#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [13]:
import pandas as pd
from A_TOOLBOX import *
import re

## Data

In [14]:
df = pd.read_csv('../data/train.csv')

## Data exploration

In [15]:
describe_df(df)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
DATA_TYPE,int64,int64,object,object,object,float64,object,object,object,object,object,object,object,float64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,912,912,19,475,6,15,35,104,8,36,91,9,158,614
CARDIN (%),100.0,100.0,2.08,52.08,0.66,1.64,3.84,11.4,0.88,3.95,9.98,0.99,17.32,67.32


In [16]:
df.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


## Data processing

In [17]:
# Tratamiento de Weight --> quitar "Kg" y convertir a float:S
lista_pesos = []
for peso in df['Weight']:
    nuevo_peso = float(peso.replace("kg",""))
    lista_pesos.append(nuevo_peso)
df['Peso_kg'] = lista_pesos

In [18]:
df.drop(columns='Weight', inplace= True)
df.head(1)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Price_euros,Peso_kg
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,749.0,2.36


In [19]:
# Ram --> Quitar GB y convertir a float:
lista_memoria = []
for memoria in df['Ram']:
    nueva_memoria = int(memoria.replace("GB",""))
    lista_memoria.append(nueva_memoria)
df['RAM_GB'] = lista_memoria

In [20]:
df.drop(columns='Ram', inplace= True)
df.head(1)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Memory,Gpu,OpSys,Price_euros,Peso_kg,RAM_GB
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,1TB HDD,AMD Radeon R7 M445,Windows 10,749.0,2.36,8


In [21]:
# Memory --> Poner todo en la misma medida y separar por tipo de memoria
df['Memory'].unique()

array(['1TB HDD', '256GB SSD', '256GB SSD +  1TB HDD',
       '512GB SSD +  2TB HDD', '128GB SSD +  1TB HDD', '128GB SSD',
       '1TB SSD', '64GB Flash Storage', '2TB HDD', '500GB HDD',
       '1.0TB Hybrid', '16GB Flash Storage', '256GB Flash Storage',
       '180GB SSD', '32GB Flash Storage', '512GB SSD',
       '512GB SSD +  1TB HDD', '256GB SSD +  500GB HDD',
       '256GB SSD +  2TB HDD', '1TB SSD +  1TB HDD', '64GB SSD',
       '128GB Flash Storage', '256GB SSD +  1.0TB Hybrid', '32GB SSD',
       '512GB Flash Storage', '240GB SSD', '256GB SSD +  256GB SSD',
       '16GB SSD', '8GB SSD', '64GB Flash Storage +  1TB HDD',
       '1.0TB HDD', '1TB HDD +  1TB HDD', '512GB SSD +  512GB SSD',
       '128GB SSD +  2TB HDD', '32GB HDD', '512GB SSD +  1.0TB Hybrid'],
      dtype=object)

In [22]:
def valores_a_GB (value):
    if 'TB' in value:
        return int(float(value.replace('TB', '').strip()) * 1024)
    elif 'GB' in value:
        return int(float(value.replace('GB', '').strip()))
    return 0

def tipo_memoria (storage, storage_type):
    if storage_type not in storage:
        return 0
    matches = re.findall(r'(\d+(?:\.\d+)?\s*TB|\d+\s*GB)\s*' + storage_type, storage)
    total_gb = sum(valores_a_GB(match) for match in matches)
    return total_gb

In [23]:
df['SSD_GB'] = df['Memory'].apply(lambda x: tipo_memoria(x, 'SSD'))
df['HDD_GB'] = df['Memory'].apply(lambda x: tipo_memoria(x, 'HDD'))
df['Flash_Storage_GB'] = df['Memory'].apply(lambda x: tipo_memoria(x, 'Flash Storage'))
df['Hybrid_GB'] = df['Memory'].apply(lambda x: tipo_memoria(x, 'Hybrid'))

In [24]:
df.drop(columns='Memory', inplace= True)
df.head(1)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Gpu,OpSys,Price_euros,Peso_kg,RAM_GB,SSD_GB,HDD_GB,Flash_Storage_GB,Hybrid_GB
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,AMD Radeon R7 M445,Windows 10,749.0,2.36,8,0,1024,0,0


In [25]:
# CPU y GPU --> extraer información en otras columnas
df['Cpu'].unique()

array(['Intel Core i7 7500U 2.7GHz',
       'Intel Pentium Quad Core N4200 1.1GHz',
       'Intel Core i5 6200U 2.3GHz', 'Intel Core i7 7700HQ 2.8GHz',
       'Intel Core i5 6300U 2.4GHz', 'Intel Core i3 6006U 2GHz',
       'Intel Pentium Quad Core N3710 1.6GHz',
       'Intel Core i7 8550U 1.8GHz', 'AMD A9-Series A9-9420 3GHz',
       'Intel Pentium Dual Core 4405U 2.1GHz',
       'Intel Core i5 8250U 1.6GHz', 'Intel Core i3 7100U 2.4GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Xeon E3-1535M v6 3.1GHz',
       'Intel Atom x5-Z8550 1.44GHz', 'AMD A10-Series 9620P 2.5GHz',
       'Intel Core i5 7300U 2.6GHz',
       'Intel Celeron Dual Core N3350 1.1GHz',
       'Intel Core i7 6500U 2.5GHz', 'Intel Core i3 6006U 2.0GHz',
       'Intel Celeron Dual Core 3205U 1.5GHz',
       'AMD A12-Series 9720P 3.6GHz',
       'Intel Celeron Dual Core N3050 1.6GHz',
       'Intel Core i3 7130U 2.7GHz', 'Intel Core i5 7300HQ 2.5GHz',
       'Intel Core i5 7200U 2.7GHz', 'Intel Core i7 7820HQ 2.9GHz

In [26]:
def extract_brand(cpu):
    return cpu.split()[0]

def extract_series_model(cpu):
    match = re.search(r'(Intel|AMD) (.*?) (\d+)', cpu)
    if match:
        return match.group(2)
    return None

def extract_speed(cpu):
    match = re.search(r'(\d+(\.\d+)?)GHz', cpu)
    if match:
        return float(match.group(1))
    return None

def extract_cores(cpu):
    if 'Quad Core' in cpu:
        return 4
    elif 'Dual Core' in cpu:
        return 2
    elif 'Hexa Core' in cpu:
        return 6
    elif 'Octa Core' in cpu:
        return 8
    else:
        return 1  # Asumiendo un solo núcleo si no se especifica


In [27]:
df['Marca_CPU'] = df['Cpu'].apply(extract_brand)
df['Modelo_Serie_CPU'] = df['Cpu'].apply(extract_series_model)
df['Velocidad_CPU_GHz'] = df['Cpu'].apply(extract_speed)
df['Cores_CPU'] = df['Cpu'].apply(extract_cores)

In [28]:
for marca in df['Marca_CPU'].unique():
    df[marca] = df['Marca_CPU'].apply(lambda x: 1 if marca in x else 0)

df.rename(columns={'Intel':'CPU_Intel','AMD':'CPU_AMD'}, inplace=True)

In [29]:
df.drop(columns='Marca_CPU',inplace=True)
df.head(1)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Gpu,OpSys,...,RAM_GB,SSD_GB,HDD_GB,Flash_Storage_GB,Hybrid_GB,Modelo_Serie_CPU,Velocidad_CPU_GHz,Cores_CPU,CPU_Intel,CPU_AMD
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,AMD Radeon R7 M445,Windows 10,...,8,0,1024,0,0,Core i7,2.7,1,1,0


In [30]:
df['Cores_CPU'].value_counts()

Cores_CPU
1    824
2     61
4     27
Name: count, dtype: int64

In [31]:
df.drop(columns='Cpu', inplace= True)
df.head(1)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Gpu,OpSys,Price_euros,...,RAM_GB,SSD_GB,HDD_GB,Flash_Storage_GB,Hybrid_GB,Modelo_Serie_CPU,Velocidad_CPU_GHz,Cores_CPU,CPU_Intel,CPU_AMD
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,AMD Radeon R7 M445,Windows 10,749.0,...,8,0,1024,0,0,Core i7,2.7,1,1,0


In [32]:
df['Gpu'].unique()

array(['AMD Radeon R7 M445', 'Intel HD Graphics 505',
       'Intel HD Graphics 520', 'Nvidia GeForce GTX 1070',
       'Nvidia GeForce GT 940MX', 'Nvidia GeForce GTX 1050',
       'Nvidia GeForce 940MX', 'Nvidia GeForce GTX 1050 Ti',
       'Intel HD Graphics 405', 'Intel UHD Graphics 620',
       'AMD Radeon 530', 'Intel HD Graphics 510', 'Intel HD Graphics 620',
       'Nvidia Quadro M2200M', 'Nvidia GeForce MX150',
       'Intel HD Graphics 400', 'Nvidia Quadro M1200',
       'Nvidia GeForce 920MX', 'Intel HD Graphics 500', 'AMD Radeon 520',
       'AMD Radeon R5 M330', 'AMD Radeon R5 M420X', 'Intel HD Graphics',
       'Intel HD Graphics 630', 'AMD Radeon R5 M430',
       'Intel Iris Pro Graphics', 'Nvidia GeForce GTX 940MX',
       'Intel HD Graphics 515', 'Nvidia GeForce GTX 1080',
       'AMD Radeon R2 Graphics', 'Nvidia GeForce 150MX', 'AMD Radeon R5',
       'Nvidia GeForce GTX 1050Ti', 'AMD Radeon R9 M385',
       'Nvidia GeForce 930MX', 'Nvidia GeForce GTX 1060',
       'Nv

In [33]:
marcas = ['AMD', 'Intel', 'Nvidia']

In [34]:
for marca in marcas:
    df[marca] = df['Gpu'].apply(lambda x: 1 if marca in x else 0)

df.rename(columns={'AMD':'GPU_AMD','Intel':'GPU_Intel','Nvidia':'GPU_Nvidia'}, inplace=True)

df['GPU_tipo_serie'] = df['Gpu'].apply(lambda x: ' '.join(x.split(' ')[1:]) if ' ' in x else None)

In [35]:
df.drop(columns='Gpu', inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 912 non-null    int64  
 1   laptop_ID          912 non-null    int64  
 2   Company            912 non-null    object 
 3   Product            912 non-null    object 
 4   TypeName           912 non-null    object 
 5   Inches             912 non-null    float64
 6   ScreenResolution   912 non-null    object 
 7   OpSys              912 non-null    object 
 8   Price_euros        912 non-null    float64
 9   Peso_kg            912 non-null    float64
 10  RAM_GB             912 non-null    int64  
 11  SSD_GB             912 non-null    int64  
 12  HDD_GB             912 non-null    int64  
 13  Flash_Storage_GB   912 non-null    int64  
 14  Hybrid_GB          912 non-null    int64  
 15  Modelo_Serie_CPU   912 non-null    object 
 16  Velocidad_CPU_GHz  912 non

In [36]:
df['ScreenResolution'].unique()

array(['1366x768', 'Full HD 1920x1080', 'IPS Panel 4K Ultra HD 3840x2160',
       'Full HD / Touchscreen 1920x1080', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'IPS Panel Touchscreen 1920x1200',
       '4K Ultra HD / Touchscreen 3840x2160', 'Touchscreen 1366x768',
       'IPS Panel Retina Display 2880x1800', 'IPS Panel 1366x768',
       'IPS Panel Quad HD+ / Touchscreen 3200x1800',
       'Quad HD+ / Touchscreen 3200x1800', '4K Ultra HD 3840x2160',
       'IPS Panel 2560x1440', '1600x900', 'Touchscreen 2560x1440',
       'IPS Panel Quad HD+ 2560x1440',
       'IPS Panel Retina Display 2304x1440', '1440x900',
       'IPS Panel Retina Display 2560x1600', 'Touchscreen 2256x1504',
       '2560x1440', 'IPS Panel Touchscreen / 4K Ultra HD 3840x2160',
       'IPS Panel Touchscreen 2560x1440', '1920x1080',
       'Touchscreen 2400x1600', 'IPS Panel Full HD 2160x1440',
       'Quad HD+ 3200x1800',
       'IPS Panel 4K Ultra HD / Touchscreen 3840x21

In [37]:
patron_resolucion = re.compile(r'(\d+)x(\d+)')

def calcular_pixeles(resolucion):
    match = patron_resolucion.search(resolucion)
    ancho, alto = map(int, match.groups())
    return ancho * alto

In [38]:
df['Num_Pixeles'] = df['ScreenResolution'].apply(calcular_pixeles)
df.head(1)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,OpSys,Price_euros,Peso_kg,...,Modelo_Serie_CPU,Velocidad_CPU_GHz,Cores_CPU,CPU_Intel,CPU_AMD,GPU_AMD,GPU_Intel,GPU_Nvidia,GPU_tipo_serie,Num_Pixeles
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Windows 10,749.0,2.36,...,Core i7,2.7,1,1,0,1,0,0,Radeon R7 M445,1049088


In [39]:
caracteristicas = ['IPS', 'HD', 'Touchscreen', '4K', 'Retina', 'Quad HD+']

def tiene_caracteristica(resolucion, caracteristica):
    return 1 if caracteristica.lower() in resolucion.lower() else 0

for caracteristica in caracteristicas:
    df[caracteristica] = df['ScreenResolution'].apply(lambda x: tiene_caracteristica(x, caracteristica))

In [40]:
df.rename(columns={'IPS':'Screen_IPS', 'HD':'Screen_HD', 'Touchscreen':'Screen_Touchscreen', '4K':'Screen_4K', 'Retina':'Screen_Retina', 'Quad HD+':'Screen_QuadHD+'}, inplace=True)

In [41]:
df.drop(columns='ScreenResolution', inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  912 non-null    int64  
 1   laptop_ID           912 non-null    int64  
 2   Company             912 non-null    object 
 3   Product             912 non-null    object 
 4   TypeName            912 non-null    object 
 5   Inches              912 non-null    float64
 6   OpSys               912 non-null    object 
 7   Price_euros         912 non-null    float64
 8   Peso_kg             912 non-null    float64
 9   RAM_GB              912 non-null    int64  
 10  SSD_GB              912 non-null    int64  
 11  HDD_GB              912 non-null    int64  
 12  Flash_Storage_GB    912 non-null    int64  
 13  Hybrid_GB           912 non-null    int64  
 14  Modelo_Serie_CPU    912 non-null    object 
 15  Velocidad_CPU_GHz   912 non-null    float64
 16  Cores_CP

In [42]:
df.to_csv('../data/train_0prog.csv')